# Нейронка на gpt-3 для векторизации описания фильмов

# import

In [15]:
from Database.Database import DataFunFilm
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim
import re
import numpy as np
from Handlers.preProc import PrepText
from tqdm import tqdm

# Загрузка базы данных

In [4]:
db = DataFunFilm()


In [117]:
table = db.getTableDF()
description = db.remakeDescriprionToArray()
print(description[0])

['шоу', 'повествует', 'о', 'героической', 'борьбе', 'сэра', 'фрэнсиса', 'грязного', 'и', 'его', 'верных', 'друзей', 'против', 'зловещего', 'тёмного', 'бога', 'в', 'сюрреалистической', '«омнивселенной».']


In [118]:
import gensim.downloader as api
rus_words = api.load('word2vec-ruscorpora-300')

In [119]:
def getWordVec(n, vectors, word):
  words_vec.append(rus_words[word])
  n += 1
  vectors += words_vec
  return n, vectors

In [120]:
sent_vec = []
n = 0
for i in range(100):
  words_vec = []
  vectors = []
  for word in description[i]:
    if word + "_NOUN" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_NOUN")
    if word + "_VERB" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_VERB")
    if word + "_DET" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_DET")
    if word + "_ADJ" in rus_words:
      n, vectors = getWordVec(n, vectors, word + "_ADJ")
  for j in range(len(vectors)):
    sent_vec.append(vectors[j] / n)

In [121]:
def getNumberMinimum(arr, arrmin):
  number = 0
  for i in range(len(arr)):
    if arr[i] < arrmin[i]:
      number += 1

  return number

In [122]:
def compareDescriptions(number):
  min = number - 1
  min_vec = 10000
  for i in range(1000):
    if (i != number):
      iter = np.abs(sent_vec[number] - sent_vec[i])
      minimum = np.abs(sent_vec[number] - sent_vec[min])
      if min_vec < getNumberMinimum(iter, minimum):
        min = i
        min_vec = getNumberMinimum(iter, minimum)
  return min

In [123]:
number = 500

min = compareDescriptions(number)

print(description[min])
print(description[number])

['праздник', 'всевозможных', 'овощных', 'блюд,', 'побуждающих', 'и', 'мясоедов', 'и', 'вегетарианцев', 'всегда', 'ставить', 'перед', 'собой', 'на', 'стол', 'овощи!', 'джейми', 'полагает,', 'что', 'самое', 'время', 'перестать', 'думать', 'об', 'овощах,', 'как', 'о', 'дополнении', 'к', 'основной', 'еде,', '—', 'овощи', 'способны', 'сами', 'стать', 'героем', 'шоу,', 'и', 'джейми', 'готов', 'показать', 'нам', 'как.', 'джейми', 'знает,', 'что', 'многие', 'из', 'нас', 'хотят', 'есть', 'поменьше', 'мяса', '—', 'кто', 'по', 'соображениям', 'здоровья,', 'кто', 'из-за', 'денег,', 'а', 'кто-то', 'из', 'гуманных', 'соображений.', 'все', 'мы', 'медленно', 'движемся', 'в', 'этом', 'направлении', '—', 'есть', 'больше', 'овощей', 'и', 'меньше', 'мяса.']
['трогательная', 'история', 'о', 'супружеской', 'чете', 'мальхотра.', 'они', 'посвятили', 'жизнь,', 'чтобы', 'вырастить', 'четырех', 'сыновей,', 'и', 'как', 'и', 'другие', 'родители', 'рассчитывали,', 'что', 'сыновья', 'станут', 'поддержкой', 'и', 'опо

In [124]:
def getWordsFromDescription(sentence):
  words = []
  for word in sentence:
    if word + "_NOUN" in rus_words:
      words.append(word + "_NOUN")
    elif word + "_VERB" in rus_words:
      words.append(word + "_VERB")
    elif word + "_DET" in rus_words:
      words.append(word + "_DET")
    elif word + "_ADJ" in rus_words:
      words.append(word + "_ADJ")
  return words

In [125]:
def anotherCompareAlgorithm(n, description):
  sentence = getWordsFromDescription(description[n])
  sentence_min = getWordsFromDescription(description[n - 2])
  min_similarity = rus_words.wmdistance(sentence, sentence_min)
  min_num = n - 2
  for i in range(100):
    if i != n:
      new_sentence = getWordsFromDescription(description[i])
      similarity = rus_words.wmdistance(sentence, new_sentence)
      if similarity < min_similarity:
        min_similarity = similarity
        min_num = i
  return min_num

In [147]:

number = 34

min_num = anotherCompareAlgorithm(number, description)


print(description[number])
print(description[min_num])
print(min_num)

['сидя', 'на\xa0автобусной', 'остановке,', 'форрест', 'гамп\xa0—\xa0не', 'очень', 'умный,', 'но\xa0добрый', 'и\xa0открытый', 'парень\xa0—\xa0рассказывает', 'случайным', 'встречным', 'историю', 'своей', 'необыкновенной', 'жизни.\n\nс', 'самого', 'малолетства', 'он\xa0страдал', 'от\xa0заболевания', 'ног,', 'и\xa0соседские', 'хулиганы', 'дразнили', 'мальчика,', 'и\xa0в', 'один', 'прекрасный', 'день', 'форрест', 'открыл', 'в\xa0себе', 'невероятные', 'способности', 'к\xa0бегу.', 'подруга', 'детства', 'дженни', 'всегда', 'его\xa0поддерживала', 'и\xa0защищала,', 'но\xa0вскоре', 'дороги', 'их\xa0разошлись']
['в', 'спектакле', 'два', 'главных', 'действующих', 'лица:', 'мальчик', 'оскар,', 'который', 'неизлечимо', 'болен,', 'и', 'розовая', 'дама,', 'одна', 'из', 'тех', 'сиделок,', 'одетых', 'в', 'розовую', 'одежду,', 'которые', 'ухаживают', 'за', 'больными', 'детьми.', 'розовая', 'дама,', 'или,', 'как', 'оскар', 'её', 'называет,', 'розовая', 'мама,', 'предлагает', 'мальчику', 'игру', '—', 'кажды

In [5]:
db = DataFunFilm()
films = db.getTableDF()

In [9]:
from nltk import word_tokenize
from nltk import download
download('punkt')  # Download data for tokenizer.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xande\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

In [12]:
def prepareTextes():
  w2v_corpus = []
  wmd_corpus = []
  for line in films["prepare_description"]:
      text = preprocess(line)

      # Add to corpus for training Word2Vec.
      w2v_corpus.append(text)

      wmd_corpus.append(text)
  return w2v_corpus, wmd_corpus

In [23]:
w2v_corpus, wmd_corpus = prepareTextes()
model = Word2Vec(w2v_corpus, workers=3, vector_size=100)

In [31]:
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(films["prepare_description"], model, num_best=10)

In [36]:
sent = films["prepare_description"][0]
print(sent)
sims = instance[films["prepare_description"][0]]
print(sims)

шоу_PROPN  повествовать_VERB о_ADP героический_ADJ борьба_NOUN сэр_NOUN фрэнсис_PROPN  грязной_ADJ и_CCONJ он_PRON верный_ADJ друг_NOUN против_ADP зловещий_ADJ темный_ADJ бог_NOUN в_ADP сюрреалистический_ADJ омнивселенной_NOUN


AttributeError: 'Word2Vec' object has no attribute 'wmdistance'